# Расчет СМО M/H2/n с H2-разогревом

### Задаем входные параметры:

In [32]:
from most_queue.theory.m_h2_h2warm import Mh2h2Warm
from most_queue.sim import smo_im
from most_queue.sim import rand_destribution as rd

import math
import time

n = 5 # число каналов
l = 1.0  # интенсивность вх потока
ro = 0.7  # коэфф загрузки
b1 = n * ro  # ср время обслуживания
b1_warm = n * 0.2  # ср время разогрева
num_of_jobs = 3000000  # число обсл заявок ИМ
b_coev = 1.1  # коэфф вариации времени обсл
b_coev_warm = 1.3  # коэфф вариации времени разогрева
buff = None
verbose = False  # не выводить пояснения при расчетах

Задаем начальные моменты времени обслуживания

In [33]:
b = [0.0] * 3
alpha = 1 / (b_coev ** 2)
b[0] = b1
b[1] = math.pow(b[0], 2) * (math.pow(b_coev, 2) + 1)
b[2] = b[1] * b[0] * (1.0 + 2 / alpha)

Задаем начальные моменты времени разогрева

In [34]:
b_w = [0.0] * 3
b_w[0] = b1_warm
alpha = 1 / (b_coev_warm ** 2)
b_w[1] = math.pow(b_w[0], 2) * (math.pow(b_coev_warm, 2) + 1)
b_w[2] = b_w[1] * b_w[0] * (1.0 + 2 / alpha)

Запускаем ИМ

In [35]:
im_start = time.process_time()

smo = smo_im.SmoIm(n, buffer=buff)  # экземпляр класса ИМ
smo.set_sources(l, 'M')  # задаем параметры вх потока (параметры, тип распределения)

gamma_params = rd.Gamma.get_mu_alpha(b)
smo.set_servers(gamma_params, 'Gamma')  # задаем параметры обслуживания

gamma_params_warm = rd.Gamma.get_mu_alpha(b_w)
smo.set_warm(gamma_params_warm, 'Gamma')  # задаем параметры разогрева

smo.run(num_of_jobs)  # запуск ИМ

# после окончания ИМ сохраняем вероятности состояний и нач моменты времени пребывания
p = smo.get_p()
w_im = smo.w

im_time = time.process_time() - im_start

Start simulation...


100%|██████████| 3000000/3000000 [00:28<00:00, 106064.64it/s]


Запуск расчета итерационным методом

In [36]:
tt_start = time.process_time()
tt = Mh2h2Warm(l, b, b_w, n, buffer=buff, verbose=verbose)  # передаем параметры в экземпляр класса

tt.run()  # запуск расчета

# после окончания расчета сохраняем вероятности состояний и нач моменты времени пребывания
p_tt = tt.get_p()
w_tt = tt.get_w()

tt_time = time.process_time() - tt_start

num_of_iter = tt.num_of_iter_  # число итераций

Вывод результатов

In [37]:
print("\nСравнение результатов расчета методом Такахаси-Таками и ИМ.\n"
              "ИМ - M/Gamma/{0:^2d} с Gamma разогревом\nТакахаси-Таками - M/H2/{0:^2d} c H2-разогревом "
              "с комплексными параметрами\n"
              "Коэффициент загрузки: {1:^1.2f}".format(n, ro))
print(f'Коэффициент вариации времени обслуживания {b_coev:0.3f}')
print(f'Коэффициент вариации времени разогрева {b_coev_warm:0.3f}')
print("Количество итераций алгоритма Такахаси-Таками: {0:^4d}".format(num_of_iter))
print("Время работы алгоритма Такахаси-Таками: {0:^5.3f} c".format(tt_time))
print("Время ИМ: {0:^5.3f} c".format(im_time))
print("{0:^25s}".format("Первые 10 вероятностей состояний СМО"))
print("{0:^3s}|{1:^15s}|{2:^15s}".format("№", "Числ", "ИМ"))
print("-" * 32)
for i in range(11):
    print("{0:^4d}|{1:^15.3g}|{2:^15.3g}".format(i, p_tt[i], p[i]))

print("\n")
print("{0:^25s}".format("Начальные моменты времени ожидания в СМО"))
print("{0:^3s}|{1:^15s}|{2:^15s}".format("№", "Числ", "ИМ"))
print("-" * 32)
for i in range(3):
    print("{0:^4d}|{1:^15.3g}|{2:^15.3g}".format(i + 1, w_tt[i], w_im[i]))


Сравнение результатов расчета методом Такахаси-Таками и ИМ.
ИМ - M/Gamma/5  с Gamma разогревом
Такахаси-Таками - M/H2/5  c H2-разогревом с комплексными параметрами
Коэффициент загрузки: 0.70
Коэффициент вариации времени обслуживания 1.100
Коэффициент вариации времени разогрева 1.300
Количество итераций алгоритма Такахаси-Таками:  26 
Время работы алгоритма Такахаси-Таками: 0.203 c
Время ИМ: 29.109 c
Первые 10 вероятностей состояний СМО
 № |     Числ      |      ИМ       
--------------------------------
 0  |    0.0215     |    0.0217     
 1  |    0.0865     |    0.0871     
 2  |     0.156     |     0.156     
 3  |     0.184     |     0.185     
 4  |     0.161     |     0.162     
 5  |     0.111     |     0.111     
 6  |    0.0783     |    0.0783     
 7  |    0.0559     |     0.056     
 8  |    0.0401     |    0.0399     
 9  |    0.0289     |    0.0285     
 10 |    0.0209     |    0.0207     


Начальные моменты времени ожидания в СМО
 № |     Числ      |      ИМ       
----